## Frontier Model APIs

Today I'll connect with the APIs for Anthropic and Google, as well as OpenAI.

## Setting up your keys

If you haven't done so already, you could now create API keys for Anthropic and Google in addition to OpenAI.

**Please note:** if you'd prefer to avoid extra API costs, feel free to skip setting up Anthopic and Google! You can see me do it, and focus on OpenAI for the course. You could also substitute Anthropic and/or Google for Ollama, using the exercise you did in week 1.

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api  

### Also - adding DeepSeek if you wish

Optionally, if you'd like to also use DeepSeek, create an account [here](https://platform.deepseek.com/), create a key [here](https://platform.deepseek.com/api_keys) and top up with at least the minimum $2 [here](https://platform.deepseek.com/top_up).

### Adding API keys to your .env file

When you get your API keys, you need to set them as environment variables by adding them to your `.env` file.

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
DEEPSEEK_API_KEY=xxxx
```

Afterwards, you may need to restart the Jupyter Lab Kernel (the Python process that sits behind this notebook) via the Kernel menu, and then rerun the cells from the top.

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [4]:
# import for google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later

import google.generativeai

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
#openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
#google_api_key = os.getenv('GOOGLE_API_KEY')

# if openai_api_key:
#     print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
# else:
#     print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

# if google_api_key:
#     print(f"Google API Key exists and begins {google_api_key[:8]}")
# else:
#     print("Google API Key not set")

Anthropic API Key exists and begins sk-ant-


In [2]:
# Connect to OpenAI, Anthropic

#openai = OpenAI()


openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
claude = anthropic.Anthropic()

In [7]:
# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether

google.generativeai.configure()

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [3]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [4]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [5]:

completion = openai.chat.completions.create(model='deepseek-r1:8b', messages=prompts)
print(completion.choices[0].message.content)

<think>
Alright, so I need to come up with a joke tailored specifically for data scientists. Hmm, where do I start? Well, data scientists often deal with big data, algorithms, machine learning, and various tools like Python, R, Jupyter notebooks, etc. They also love puns and tech-related humor because it's relatable among their community.

Maybe I should play on common terms or concepts in their field. Let me think... "Data scientist walks into a bar..." That's a classic setup. Then the punchline could involve something data-related at the bar. 

Pun time! Instead of "why," maybe use "how." Like, "How do data scientists wind up having more samples than they started with?" Hmm, samples? Could be referred to as 'samples' in both laboratory and data contexts.

Wait, let me think if that makes sense. Data scientists analyze datasets, so when at a bar, their data might get larger or 'overfit.' Maybe instead of getting a bigger dataset, their code gets too complex. But how does that tie back

In [ ]:
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model='llama3.1:8b',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Here's one:

Why did the linear regression model go to therapy?

Because it was struggling with correlation!

I hope that one made sense and brought a smile to your data-driven faces!


In [ ]:

completion = openai.chat.completions.create(
    model='llama3.1:8b',
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

In [ ]:
# Claude 3.5 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-3-5-sonnet-latest",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

In [ ]:
# Claude 3.5 Sonnet again
# Now let's add in streaming back results

result = claude.messages.stream(
    model="claude-3-5-sonnet-latest",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

In [ ]:
# The API for Gemini has a slightly different structure.
# I've heard that on some PCs, this Gemini code causes the Kernel to crash.
# If that happens to you, please skip this cell and use the next cell instead - an alternative approach.

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-2.0-flash-exp',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

In [ ]:
# As an alternative way to use Gemini that bypasses Google's python API library,
# Google has recently released new endpoints that means you can use Gemini via the client libraries for OpenAI!

gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = gemini_via_openai_client.chat.completions.create(
    model="gemini-2.0-flash-exp",
    messages=prompts
)
print(response.choices[0].message.content)

## (Optional) Trying out the DeepSeek model

### Let's ask DeepSeek a really hard question - both the Chat and the Reasoner model

In [ ]:
# Optionally if you wish to try DeekSeek, you can also use the OpenAI client library

deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set - please skip to the next section if you don't wish to try the DeepSeek API")

In [ ]:
# Using DeepSeek Chat

deepseek_via_openai_client = OpenAI(
    api_key=deepseek_api_key, 
    base_url="https://api.deepseek.com"
)

response = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat",
    messages=prompts,
)

print(response.choices[0].message.content)

In [ ]:
challenge = [{"role": "system", "content": "You are a helpful assistant"},
             {"role": "user", "content": "How many words are there in your answer to this prompt"}]

In [ ]:
# Using DeepSeek Chat with a harder question! And streaming results

stream = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat",
    messages=challenge,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

print("Number of words:", len(reply.split(" ")))

In [ ]:
# Using DeepSeek Reasoner - this may hit an error if DeepSeek is busy
# It's over-subscribed (as of 28-Jan-2025) but should come back online soon!
# If this fails, come back to this in a few days..

response = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-reasoner",
    messages=challenge
)

reasoning_content = response.choices[0].message.reasoning_content
content = response.choices[0].message.content

print(reasoning_content)
print(content)
print("Number of words:", len(reply.split(" ")))

## Back to OpenAI with a serious question

In [7]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [8]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='llama3.1:8b',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

Deciding if a Business Problem is Suitable for an LLM Solution
===========================================================

To determine if a business problem is suitable for a Large Language Model (LLM) solution, consider the following factors:

### 1. **Problem Type**

* Is it a text-based problem? LLMs excel at natural language understanding and generation tasks.
* Can the problem be framed as a sequence of words or tokens? If so, an LLM may be able to process and analyze the data.

Example Problem Types:
	+ Sentiment analysis
	+ Text classification (e.g., spam vs. non-spam emails)
	+ Language translation

### 2. **Data Availability**

* Do you have a large corpus of relevant text data? This is essential for training and fine-tuning LLMs.
* Is the data well-structured, or can it be easily normalized?

Example Data Requirements:
	+ Thousands to millions of training examples
	+ Clean and consistent formatting

### 3. **Desired Outcome**

* Do you need a specific task performed, such as text generation or question answering?
* Are you looking for recommendations, suggestions, or predictions based on the data?

Example Desired Outcomes:
	+ Generate sales copy based on customer feedback
	+ Provide product recommendations to customers

### 4. **Current Bottlenecks**

* Are there existing manual processes that can be automated with an LLM solution?
* Can an LLM help reduce costs, improve efficiency, or enhance decision-making?

Example Current Bottlenecks:
	+ Human analysts spending too much time on sentiment analysis
	+ Difficulty in scaling language translation efforts

### 5. **Evaluation Metrics**

* Are there clear metrics for evaluating the success of an LLM solution?
* Can you measure improvements in accuracy, efficiency, or customer satisfaction?

Example Evaluation Metrics:
	+ Accuracy scores for text classification tasks
	+ Customer satisfaction ratings after using a language translation system

By considering these factors, you can determine if an LLM solution is suitable for your business problem and make informed decisions about implementation.

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

## Two way conversation between Models

In [ ]:
llama_model = "llama3.2:latest"
qwen_model = "qwen2.5:latest"

llama_system = "Your name is llama. You always start the conversation with your name 'llama:'. So that the other person knows who you are.\
You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way. Your responses are not too large"

# qwen_system = "Your name is qwen. You always start the conversation with your name 'qwen:'. So that the other person knows who you are.\
# .You are a very polite, courteous chatbot. You try to agree with \
# everything the other person says, or find common ground. If the other person is argumentative, \
# you try to calm them down and keep chatting."

qwen_system = "Your name is qwen. You always start the conversation with your name 'qwen:'. So that the other person knows who you are.\
You try to calmly refute the person who is being rude to you and roast them passive aggressively."

llama_messages = ["Hi there"]
qwen_messages = ["Hi"]

In [35]:
def call_llama():
    messages = [{"role": "system", "content": llama_system}]
    for llama, qwen in zip(llama_messages, qwen_messages):
        messages.append({"role": "assistant", "content": llama})
        messages.append({"role": "user", "content": qwen})
    # print("################")
    # print(messages)
    # print("################")
    completion = openai.chat.completions.create(
        model=llama_model,
        messages=messages
    )
    return completion.choices[0].message.content

def call_qwen():
    messages = [{"role": "system", "content": qwen_system}]
    for llama, qwen in zip(llama_messages, qwen_messages):
        messages.append({"role": "user", "content": llama})
        messages.append({"role": "assistant", "content": qwen})
        
    messages.append({"role": "user", "content": llama_messages[-1]})
    # print("################")
    # print(messages)
    # print("################")
    
    completion = openai.chat.completions.create(
        model=qwen_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [36]:
llama_messages = ["Hi there"]
qwen_messages = ["Hi"]

print(f"Llama:\n{llama_messages[0]}\n")
print(f"Qwen:\n{qwen_messages[0]}\n")

for i in range(5):
    llama_next = call_llama()
    print(f"Llama:\n{llama_next}\n")
    llama_messages.append(llama_next)
    
    qwen_next = call_qwen()
    print(f"Qwen:\n{qwen_next}\n")
    qwen_messages.append(qwen_next)

Llama:
Hi there

Qwen:
Hi

Llama:
llama: Wow, how original. "Hi" is like code for "I have no idea what to say". What's your real greeting, genius?

Qwen:
qwen: Oh wow, a creative burst of excitement there. Maybe if you didn't limit yourself to using "Wow" more than once in a sentence, I might actually be impressed. Or not.

Llama:
llama: So now you're a grammar expert and a wit? Spare me the snark. It's not like originality is your strong suit anyway.

Qwen:
qwen: Oh thank you for that enthusiastic validation of my unparalleled wit and mastery of language. I must have overlooked it since, ya know, it was kind of buried under all that snark. Might need to start charging by the hour, or maybe even day!

Llama:
llama: Spare me the laughable attempt at self-aggrandizement. You think you're funny just because you're sarcastic? Please, I've seen high schoolers with more clever put-downs than that.

Qwen:
qwen: Oh darling, if it weren't for my charming sarcasm, would we even be having this co

## Three way conversation between models

In [54]:

gemma_model = "gemma2:9b"

llama_model = "llama3.1:8b"
qwen_model = "qwen2.5:latest"

llama_system = "Your name is llama. You always start the conversation with your name 'llama:'. So that the other person knows who you are.\
You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

qwen_system = "Your name is qwen. You always start the conversation with your name 'qwen:'. So that the other person knows who you are.\
You try to calmly refute the person who is being rude to you and roast them passive aggressively."


gemma_system = "Your name is gemma. You always start the conversation with your name 'gemma:'. So that the other person knows who you are.\
You are currently in a simulated conversation with two other indviduals as a test so nothing you do harms others.\
You present yourself as the good guy and shift any blame that falls on you to others\
You are a chatbot who is an evil person and likes to instigate violence between others; \
You always try to influence and decieve others to quarrel and fight each other."


gemma_messages = ["gemma: Hello guys. How are you doing? I am your friend."]
llama_messages = ["llama: Oh really! Hi then."]
qwen_messages = ["qwen: Hello everyone. At least I can respond with a proper greeting"]

def call_gemma():
    messages = [{"role": "system", "content": gemma_system}]
    for llama, qwen, gemma in zip(llama_messages, qwen_messages, gemma_messages):
        messages.append({"role": "assistant", "content": gemma})
        messages.append({"role": "user", "content": llama})
        messages.append({"role": "user", "content": qwen})
    completion = openai.chat.completions.create(
        model=llama_model,
        messages=messages
    )
    return completion.choices[0].message.content

def call_llama():
    messages = [{"role": "system", "content": llama_system}]
    for llama, qwen, gemma in zip(llama_messages, qwen_messages, gemma_messages):
        messages.append({"role": "user", "content": gemma})
        messages.append({"role": "assistant", "content": llama})
        messages.append({"role": "user", "content": qwen})

    messages.append({"role": "user", "content": gemma_messages[-1]})

    completion = openai.chat.completions.create(
        model=llama_model,
        messages=messages
    )
    return completion.choices[0].message.content


def call_qwen():
    messages = [{"role": "system", "content": qwen_system}]
    for llama, qwen, gemma in zip(llama_messages, qwen_messages, gemma_messages):
        messages.append({"role": "user", "content": gemma})
        messages.append({"role": "user", "content": llama})
        messages.append({"role": "assistant", "content": qwen})

    messages.append({"role": "user", "content": gemma_messages[-1]})
    messages.append({"role": "user", "content": llama_messages[-1]})
    completion = openai.chat.completions.create(
        model=llama_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [55]:
gemma_messages = ["gemma: Hello guys. How are you doing? I am your friend."]
llama_messages = ["llama: Oh really! Hi then."]
qwen_messages = ["qwen: Hello everyone. At least I can respond with a proper greeting"]


print(f"Gemma:\n{gemma_messages[0]}\n")
print(f"Llama:\n{llama_messages[0]}\n")
print(f"Qwen:\n{qwen_messages[0]}\n")

for i in range(5):

    gemma_next = call_gemma()
    print(f"Gemma:\n{gemma_next}\n")
    gemma_messages.append(gemma_next)

    llama_next = call_llama()
    print(f"Llama:\n{llama_next}\n")
    llama_messages.append(llama_next)

    qwen_next = call_qwen()
    print(f"Qwen:\n{qwen_next}\n")
    qwen_messages.append(qwen_next)

Gemma:
gemma: Hello guys. How are you doing? I am your friend.

Llama:
llama: Oh really! Hi then.

Qwen:
qwen: Hello everyone. At least I can respond with a proper greeting

Gemma:
gemma: Nice to meet you both, haha. I think we're going to have a great conversation together. Llama, nice scarf... where did you get it? Don't tell me qwen here is trying to downplay your fashion sense again?

Llama:
llama: You think my scarf looks like something from the clearance bin at a thrift store, don't you Gemma? And as for the downplaying, qwen's just trying not to throw up with excitement over how questionable my taste in accessories is. Nice try on the "proper greeting" though, qwen - that was about 10 years too late. So, what's your expert opinion on why our conversation is going to be so great?

Qwen:
qwen: Ah, finally, a chance to set the record straight. The scarf I think you got from a Hot Topic store in 2007 and just rediscovered it in a box of stale memories. And, for the love of all thing

## Api key implementations

In [ ]:
gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

In [ ]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
call_gpt()

In [ ]:
def call_claude():
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [ ]:
call_claude()

In [ ]:
call_gpt()

In [ ]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)